## Default Setting

In [26]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
import sqlite3
from sklearn.feature_extraction.text import TfidfVectorizer
from krwordrank.word import KRWordRank

        
# 🔹 제외할 영어 관사(articles) 목록
ARTICLES = {"a", "an", "the"}

# 🔹 한글 조사 목록 (단어 끝에 붙는 경우 의미 분석 방해 → 제거 대상)
PARTICLES = {
    "은",
    "는",
    "이",
    "가",
    "을",
    "를",
    "에",
    "에서",
    "와",
    "과",
    "도",
    "의",
    "한",
    "로",
    "으로",
    "하고",
    "및",
    "등",
    "까지",
    "부터",
    "만",
    "보다",
    "처럼",
    "같이",
    "께서",
}

## URL Parsing

In [27]:
def get_naver_economy_news_urls_from_list(pages=5, max_urls=50):
    base_url = "https://news.naver.com/main/list.naver"
    headers = {"User-Agent": "Mozilla/5.0"}
    all_links = set()

    for page in range(1, pages + 1):
        params = {
            "mode": "LSD",
            "mid": "shm",
            "sid1": "101",  # 경제 뉴스 섹션
            "page": str(page)
        }
        response = requests.get(base_url, headers=headers, params=params)
        print(f"{page}페이지 응답 코드: {response.status_code}")

        if response.status_code != 200:
            print(f"{page}페이지 요청 실패")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        for a in soup.select("dt > a"):
            href = a.get("href")
            if href and href.startswith("https://n.news.naver.com"):
                all_links.add(href)
                if len(all_links) >= max_urls:
                    print(f"최대 {max_urls}개 링크 수집 완료, 수집 종료")
                    return list(all_links)

    print(f"총 {len(all_links)}개의 중복 없는 링크를 수집했습니다.")
    return list(all_links)

## Word crawling

In [28]:
def get_news_text(url):
    """
    네이버 뉴스 기사 URL로부터 본문 텍스트를 크롤링하는 함수
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    # 🔹 HTTP GET 요청
    response = requests.get(url, headers=headers)
  
    # 🔸 응답 상태 코드 확인
    if response.status_code != 200:
        print("❌ 뉴스 기사 크롤링 실패:", response.status_code)
        return ""

    # 🔹 HTML 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 🔸 네이버 뉴스 본문이 있는 div 선택 (2023년 이후 'newsct_article' 사용)
    article_body = soup.select_one("div#newsct_article")

    if article_body:
        # 🔹 줄바꿈 포함하여 텍스트 추출
        return article_body.get_text(strip=True, separator="\n")
    else:
        print("❌ 뉴스 본문을 찾을 수 없음")
        return ""

## Remove particles

In [29]:
def remove_particles(word):
    """
    한글 단어에서 조사(조사 목록에 포함된 단어)를 제거하는 함수
    예) "전산장애로" → "전산장애"
    """
    # 정규표현식을 통해 단어 끝에 붙은 조사 제거
    pattern = r"(" + "|".join(PARTICLES) + r")$"
    return re.sub(pattern, "", word)


def extract_words_okt(text):
    """
    본문에서 명사구 추출하고 조사 제거,
    중복 없이 처리하지 않고 등장 순서 그대로 반환
    """
    okt = Okt()
    nouns = okt.phrases(text)  # 명사구 추출

    filtered = [remove_particles(n) for n in nouns if n.strip()]
    return filtered  # 중복 제거 없이 그대로 반환




## 기사 내 키워드 추출

In [30]:
def extract_keywords(text_list, top_n=20):
    """
    krwordrank를 이용한 키워드 추출
    """
    wordrank_extractor = KRWordRank(
        min_count=2,     # 최소 등장 횟수
        max_length=10,   # 키워드 최대 길이
        verbose=True     # 진행 상황 출력
    )
    
    keywords, rank, graph = wordrank_extractor.extract(
        text_list,
        beta=0.85,       # 페이지랭크 계수
        max_iter=10      # 반복 횟수
    )

    sorted_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
    return sorted_keywords[:top_n]

## Print Word List

In [31]:
def print_words_in_rows(words, words_per_row=10):
    """
    단어 리스트를 지정된 개수(words_per_row)만큼 한 줄에 출력
    """
    for i in range(0, len(words), words_per_row):
        print(", ".join(words[i : i + words_per_row]))

## DB Scanning

In [32]:
# ✅ DB 조회 함수
def find_description_from_db(conn, term_input):
    cursor = conn.cursor()
    query = "SELECT * FROM dictionary WHERE term = ?"
    cursor.execute(query, (term_input,))
    result = cursor.fetchone()
    if result:
        return result[3]  # description 컬럼
    else:
        return None

    # ✅ 메인 실행 함수

def extract_and_explain(url, db_path):
    text = get_news_text(url)
    
    words_Konpy = extract_words_okt(text)
    words_Konpy = list(dict.fromkeys(words_Konpy))  # 중복 제거 (순서 유지)

    conn = sqlite3.connect(db_path)
    found = False  # ✅ 초기화

    print("🔍 기사에서 등장한 경제 용어 설명 Konlpy:\n")
    

    for word in words_Konpy:
        desc = find_description_from_db(conn, word)
        if desc:
            found = True
            print(f"📌 {word}: {desc}\n")       

    conn.close()

    if not found:
        print("📝 기사 내에서 설명 가능한 경제 용어를 찾지 못했습니다.")

## 크롤링 & 단어 matching test

In [36]:
# ✅ 실행
if __name__ == "__main__":
    db_file = r"C:\Users\wjdgh\OneDrive\바탕 화면\2025-1\2025졸업작품\Parsing&crawling\dictionary_test.db"
    
    # 1. 뉴스 URL 수집
    news_urls = get_naver_economy_news_urls_from_list(1,max_urls=20)
    
    # 2. 각 뉴스 URL에 대해 처리
for url in news_urls:
    print(f"\n📄 기사 URL: {url}")
    
    text = get_news_text(url)
    if not text.strip():
        print("❌ 기사 본문 없음")
        continue
    
    keywords = extract_keywords([text], top_n=3)

    print("📌 키워드:")
    for word, score in keywords:
        print(f"- {word}: {round(score, 4)}")


   # extract_and_explain(url, db_file)

1페이지 응답 코드: 200
최대 20개 링크 수집 완료, 수집 종료

📄 기사 URL: https://n.news.naver.com/mnews/article/277/0005614737?sid=101
scan vocabs ... 
num vocabs = 178
done = 10
📌 키워드:
- 소스: 4.8866
- 외식: 3.0009
- 제조: 2.6365

📄 기사 URL: https://n.news.naver.com/mnews/article/015/0005150905?sid=101
scan vocabs ... 
num vocabs = 381
done = 10
📌 키워드:
- 디어유: 3.6273
- 중국: 3.1568
- 제공: 3.0185

📄 기사 URL: https://n.news.naver.com/mnews/article/032/0003379274?sid=101
scan vocabs ... 
num vocabs = 163
done = 10
📌 키워드:
- 정책: 3.119
- ‘AI: 2.6374
- 이재명: 2.572

📄 기사 URL: https://n.news.naver.com/mnews/article/001/0015476817?sid=101
scan vocabs ... 
num vocabs = 391
done = 10
📌 키워드:
- 장관: 5.4358
- 후보자: 4.1748
- 산업: 4.0272

📄 기사 URL: https://n.news.naver.com/mnews/article/055/0001270667?sid=101
scan vocabs ... 
num vocabs = 202
done = 10
📌 키워드:
- 서울: 5.871
- 아파트: 3.7283
- 평균: 3.2055

📄 기사 URL: https://n.news.naver.com/mnews/article/009/0005516494?sid=101
scan vocabs ... 
num vocabs = 220
done = 10
📌 키워드:
- 예산: 2.8395
- 올해: 2